# TensorFlow 2.0

In [1]:
import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets 

## Hyperparameter Tunning
- 하이퍼 파라미터는 주로 패키지 바로 밑에 작성

In [21]:
num_epochs = 1
batch_size = 64

learning_rate = 0.001

dropout_rate = 0.7

input_shape = (28, 28, 1)
num_classes = 10

## Preprocess

In [15]:
(train_x, train_y), (test_x, test_y) = datasets.mnist.load_data()

In [16]:
train_x.shape,train_y.shape

((60000, 28, 28), (60000,))

In [17]:
# 차원 추가
train_x = train_x[..., tf.newaxis]
test_x = test_x[..., tf.newaxis]

# 정규화
train_x = train_x / 255.
test_x = test_x / 255.

In [18]:
train_x.shape,train_y.shape

((60000, 28, 28, 1), (60000,))

## Build Model

In [19]:
inputs = layers.Input(input_shape)
net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [20]:
# Model is the full model w/o custom layers
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='sparse_categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

## Training

In [22]:
model.fit(train_x, train_y,   
             batch_size=batch_size, 
             shuffle=True)

model.evaluate(test_x, test_y, batch_size=batch_size)

157/157 [==============================] - 3s 17ms/step - loss: 0.0564 - accuracy: 0.9819


[0.056441761553287506, 0.9818999767303467]

# PyTorch

In [23]:
import torch

# 모델 설계를 위한 라이브러리
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

from torchvision import datasets, transforms

In [24]:
seed = 1 # 랜덤값 고정

lr = 0.001 # 러닝레이트
momentum = 0.5

batch_size = 64 
test_batch_size = 64

epochs = 5

no_cuda = False     # cpu 사용할때 False
log_interval = 100  # 100번 돌때마다 log를 보여줌

## Model

In [25]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__() 
        self.conv1 = nn.Conv2d(1, 20, 5, 1) # in채널 out채널을 둘다 적어줘야함
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

## Preprocess

In [26]:
torch.manual_seed(seed)

use_cuda = not no_cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}


# tensor에서 데이터를 불러오고 나누는 부분
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True, **kwargs)



test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=test_batch_size, shuffle=True, **kwargs)

0it [00:00, ?it/s]Downloading http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz to ../data/MNIST/raw/train-images-idx3-ubyte.gz
 98%|█████████▊| 9682944/9912422 [00:04<00:00, 4044373.20it/s]Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw

0it [00:00, ?it/s]Downloading http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz to ../data/MNIST/raw/train-labels-idx1-ubyte.gz

 57%|█████▋    | 16384/28881 [00:00<00:00, 87828.61it/s]

0it [00:00, ?it/s]Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


  0%|          | 0/1648877 [00:00<?, ?it/s]

  1%|          | 16384/1648877 [00:00<00:20, 77896.37it/s]

  3%|▎         | 49152/1648877 [00:00<00:17, 93781.24it/s]

  6%|▌         | 98304/1648877 [00:01<00:13, 115361.16it/s]

 13%|█▎        | 212992/1648877 [00:01<00:09, 153323.53it/s]

 26%|██▋       | 434176/1648877 [00:01<00:05, 207214.47it/s]

 53%|█████▎    | 868352/1648877 [00:01<00:02, 285809.91it/s]


0it [00:00, ?it

## Optimization

In [27]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

32768it [01:43, 317.10it/s]  
1654784it [01:42, 16115.09it/s] 
8192it [01:40, 81.18it/s]               


## Training

In [ ]:
for epoch in range(1, epochs + 1):
    # Train Mode
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()  # backpropagation 계산하기 전에 0으로 기울기 계산
        output = model(data)
        loss = F.nll_loss(output, target)  # https://pytorch.org/docs/stable/nn.html#nll-loss
        loss.backward()  # 계산한 기울기를 
        optimizer.step()  

        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    
    # Test mode
    model.eval()  # batch norm이나 dropout 등을 train mode 변환
    test_loss = 0
    correct = 0
    with torch.no_grad():  # autograd engine, 즉 backpropagatin이나 gradient 계산 등을 꺼서 memory usage를 줄이고 속도를 높임
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()  # pred와 target과 같은지 확인

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))